In [1]:
import openreview
from deepdiff import DeepDiff
from pprint import pprint
from __future__ import print_function

guest_client = openreview.Client(baseurl="https://api.openreview.net")

def sort_by_tcdate(notes):
    if type(notes[0]) is dict:
        return list(sorted(notes, key=lambda x: x['tcdate']))
    else:
        return list(sorted(notes, key=lambda x: x.tcdate))

def print_metadata(note):
    if type(note) is dict:
        print(
            note['id'], note['forum'], 
            note['original'] if 'original' in note else None, 
            note['referent'] if 'referent' in note else None, 
            note['cdate'], note['tcdate'], note['tmdate'], note['invitation']
        )
    else:
        print(
            note.id, note.forum, note.original, note.referent,
            note.cdate, note.tcdate, note.tmdate, note.invitation
        )

In [2]:
VENUE = "ICLR.cc/2022/Conference"

In [3]:
# get all blind submissions
forum_notes = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2627


### Referent submission and comment notes

In [4]:
# get a blind submission
forum_note = forum_notes[0]
print('id forum original referent cdate tcdate tmdate invitation')
print_metadata(forum_note)
print(forum_note)

id forum original referent cdate tcdate tmdate invitation
g1SzIRLQXMM g1SzIRLQXMM -iEEvvwzZ1k None 1632965333637 1633409729580 1647379304182 ICLR.cc/2022/Conference/-/Blind_Submission
{'cdate': 1632965333637,
 'content': {'_bibtex': '@inproceedings{\n'
                        'geiger2022wiring,\n'
                        'title={Wiring Up Vision: Minimizing Supervised '
                        'Synaptic Updates Needed to Produce a Primate Ventral '
                        'Stream},\n'
                        'author={Franziska Geiger and Martin Schrimpf and '
                        'Tiago Marques and James J. DiCarlo},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2022},\n'
                        'url={https://openreview.net/forum?id=g1SzIRLQXMM}\n'
                        '}',
             'abstract': 'After training on large datasets, certain deep '
                         

In [5]:
# get the referent submission (forum) note and comment notes for this blind submission
this_forum_notes = sort_by_tcdate(list(openreview.tools.iterget_notes(guest_client, forum=forum_note.id)))
print('#NotesThisBlindSubmissionForum', len(this_forum_notes))

print('id forum original referent cdate tcdate tmdate invitation')
for note in this_forum_notes:
    print_metadata(note)
    if note.id == forum_note.id:
        print(DeepDiff(note, forum_note, ignore_order=True))
    else:
        comment_note = note
assert forum_note.id in [note.id for note in this_forum_notes]

#NotesThisBlindSubmissionForum 17
id forum original referent cdate tcdate tmdate invitation
g1SzIRLQXMM g1SzIRLQXMM -iEEvvwzZ1k None 1632965333637 1633409729580 1647379304182 ICLR.cc/2022/Conference/-/Blind_Submission
{}
8XgiSituROK g1SzIRLQXMM None None 1635688886814 1635688886814 1638239389734 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
6QcCfqCHr1p g1SzIRLQXMM None None 1635870614541 1635870614541 1637684442068 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
VGl7TQOdxrG g1SzIRLQXMM None None 1635879491225 1635879491225 1635879491225 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
RzRdO_6tQT g1SzIRLQXMM None None 1636361229806 1636361229806 1636361229806 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
4bUxFRU-wKY g1SzIRLQXMM None None 1637203608775 1637203608775 1637203608775 ICLR.cc/2022/Conference/Paper4724/-/Official_Comment
mGieKDTWL9Y g1SzIRLQXMM None None 1637534773054 1637534773054 1637534773054 ICLR.cc/2022/Conference/Paper4724/-/Official_Comment
hDwV3d3GCu

### Referent submission and its references

In [6]:
print("forum id\t", forum_note.id)
print("original id\t", forum_note.original)

forum id	 g1SzIRLQXMM
original id	 -iEEvvwzZ1k


In [7]:
# original submission and its revisions
submission_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=True))
print('#References:', len(submission_references_oT))

has_original = False
print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oT:
    assert ref.original == forum_note.original
    print_metadata(ref)
    if ref.id == forum_note.original:
        print("above is the original submission note")
        has_original = True
    # TODO check that the original has the same contents as the forum
    # TODO only need one of this and the forum note
if not has_original:
    print("MISSING THE ORIGINAL NOTE")

#References: 4
id forum original referent cdate tcdate tmdate invitation
KzChITSd8y g1SzIRLQXMM -iEEvvwzZ1k g1SzIRLQXMM 1632965333637 1632965333637 1632965333637 ICLR.cc/2022/Conference/-/Blind_Submission
tUsb6wBeye g1SzIRLQXMM -iEEvvwzZ1k g1SzIRLQXMM 1632965333637 1633468186623 1633468186623 ICLR.cc/2022/Conference/-/Blind_Submission
Zg1206MRod g1SzIRLQXMM -iEEvvwzZ1k g1SzIRLQXMM 1632965333637 1637534503916 1637534503916 ICLR.cc/2022/Conference/-/Blind_Submission
S6fxOmK0Z9 g1SzIRLQXMM -iEEvvwzZ1k g1SzIRLQXMM 1632965333637 1647379304182 1647379304182 ICLR.cc/2022/Conference/-/Blind_Submission
MISSING THE ORIGINAL NOTE


In [8]:
# this seems to return one of the above revisions, which is Code
guest_client.get_all_references(referent=forum_note.original, original=True)

[]

In [9]:
guest_client.get_all_references(referent=forum_note.original, original=False)

[]

In [10]:
print(forum_note)

{'cdate': 1632965333637,
 'content': {'_bibtex': '@inproceedings{\n'
                        'geiger2022wiring,\n'
                        'title={Wiring Up Vision: Minimizing Supervised '
                        'Synaptic Updates Needed to Produce a Primate Ventral '
                        'Stream},\n'
                        'author={Franziska Geiger and Martin Schrimpf and '
                        'Tiago Marques and James J. DiCarlo},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2022},\n'
                        'url={https://openreview.net/forum?id=g1SzIRLQXMM}\n'
                        '}',
             'abstract': 'After training on large datasets, certain deep '
                         'neural networks are surprisingly good models of the '
                         'neural mechanisms of adult primate visual object '
                         'recognition. Nevertheless,

In [11]:
print(submission_references_oT[0]) # original

{'cdate': 1632965333637,
 'content': {'_bibtex': '@inproceedings{\n'
                        'geiger2022wiring,\n'
                        'title={Wiring Up Vision: Minimizing Supervised '
                        'Synaptic Updates Needed to Produce a Primate Ventral '
                        'Stream},\n'
                        'author={Franziska Geiger and Martin Schrimpf and '
                        'Tiago Marques and James J. DiCarlo},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2022},\n'
                        'url={https://openreview.net/forum?id=g1SzIRLQXMM}\n'
                        '}',
             'abstract': 'After training on large datasets, certain deep '
                         'neural networks are surprisingly good models of the '
                         'neural mechanisms of adult primate visual object '
                         'recognition. Nevertheless,

In [12]:
print(submission_references_oT[1]) # revision

{'cdate': 1632965333637,
 'content': {'_bibtex': '@inproceedings{\n'
                        'geiger2022wiring,\n'
                        'title={Wiring Up Vision: Minimizing Supervised '
                        'Synaptic Updates Needed to Produce a Primate Ventral '
                        'Stream},\n'
                        'author={Franziska Geiger and Martin Schrimpf and '
                        'Tiago Marques and James J. DiCarlo},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2022},\n'
                        'url={https://openreview.net/forum?id=g1SzIRLQXMM}\n'
                        '}',
             'abstract': 'After training on large datasets, certain deep '
                         'neural networks are surprisingly good models of the '
                         'neural mechanisms of adult primate visual object '
                         'recognition. Nevertheless,

### A referent comment note and its references

In [13]:
comment_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=True))
print('#References:', len(comment_references_oT))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oT:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
_hixtLKKvWw g1SzIRLQXMM None 0zSOgdUBpLa 1642697146697 1642697146697 1642697146697 ICLR.cc/2022/Conference/Paper4724/-/Decision


In [14]:
comment_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=False))
print('#References:', len(comment_references_oF))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oF:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
_hixtLKKvWw g1SzIRLQXMM None 0zSOgdUBpLa 1642697146697 1642697146697 1642697146697 ICLR.cc/2022/Conference/Paper4724/-/Decision


In [15]:
print(comment_note)

{'cdate': 1642697146697,
 'content': {'comment': 'This paper experiments with what is required for a '
                        'deep neural network to be similar to the visual '
                        'activity in the ventral stream (as judged by the '
                        'brainscore benchmark). The authors have several '
                        'interesting contributions, such as showing that a '
                        'small number of supervised updates are required to '
                        'predict most of the variance in the brain activity, '
                        'or that models with randomized synaptic weights can '
                        'also predict a significant portion of the variance. '
                        'These different points serve to better connect deep '
                        'learning to important questions in neuroscience and '
                        'the presence of the paper at ICLR would create good '
                        'questions. The di

In [16]:
print(comment_references_oT[0])

{'cdate': 1642697146697,
 'content': {'comment': 'This paper experiments with what is required for a '
                        'deep neural network to be similar to the visual '
                        'activity in the ventral stream (as judged by the '
                        'brainscore benchmark). The authors have several '
                        'interesting contributions, such as showing that a '
                        'small number of supervised updates are required to '
                        'predict most of the variance in the brain activity, '
                        'or that models with randomized synaptic weights can '
                        'also predict a significant portion of the variance. '
                        'These different points serve to better connect deep '
                        'learning to important questions in neuroscience and '
                        'the presence of the paper at ICLR would create good '
                        'questions. The di

In [17]:
print(comment_references_oF[0])

{'cdate': 1642697146697,
 'content': {'comment': 'This paper experiments with what is required for a '
                        'deep neural network to be similar to the visual '
                        'activity in the ventral stream (as judged by the '
                        'brainscore benchmark). The authors have several '
                        'interesting contributions, such as showing that a '
                        'small number of supervised updates are required to '
                        'predict most of the variance in the brain activity, '
                        'or that models with randomized synaptic weights can '
                        'also predict a significant portion of the variance. '
                        'These different points serve to better connect deep '
                        'learning to important questions in neuroscience and '
                        'the presence of the paper at ICLR would create good '
                        'questions. The di

## Other exploration

### Get notes with direct or all replies

In [18]:
# get all blind submissions, each with its direct replies
forum_notes_dr = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='directReplies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2627


In [19]:
# get all blind submissions, each with its replies
forum_notes_r = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='replies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2627


In [20]:
# the first blind submission with direct replies
forum_note = forum_notes_dr[0]
print('#directReplies:', len(forum_note.details['directReplies']))
for reply in sort_by_tcdate(forum_note.details['directReplies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#directReplies: 6
8XgiSituROK g1SzIRLQXMM None None 1635688886814 1635688886814 1638239389734 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
6QcCfqCHr1p g1SzIRLQXMM None None 1635870614541 1635870614541 1637684442068 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
VGl7TQOdxrG g1SzIRLQXMM None None 1635879491225 1635879491225 1635879491225 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
RzRdO_6tQT g1SzIRLQXMM None None 1636361229806 1636361229806 1636361229806 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
OzJQ7cizdH0 g1SzIRLQXMM None None 1638307802121 1638307802121 1638307802121 ICLR.cc/2022/Conference/Paper4724/-/Official_Comment
0zSOgdUBpLa g1SzIRLQXMM None None 1642697146697 1642697146697 1642697146697 ICLR.cc/2022/Conference/Paper4724/-/Decision

g1SzIRLQXMM g1SzIRLQXMM -iEEvvwzZ1k None 1632965333637 1633409729580 1647379304182 ICLR.cc/2022/Conference/-/Blind_Submission
{'cdate': 1632965333637,
 'content': {'_bibtex': '@inproceedings{\n'
                        '

In [21]:
# the first blind submission with all replies
forum_note = forum_notes_r[0]
print('#replies:', len(forum_note.details['replies']))
for reply in sort_by_tcdate(forum_note.details['replies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#replies: 16
8XgiSituROK g1SzIRLQXMM None None 1635688886814 1635688886814 1638239389734 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
6QcCfqCHr1p g1SzIRLQXMM None None 1635870614541 1635870614541 1637684442068 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
VGl7TQOdxrG g1SzIRLQXMM None None 1635879491225 1635879491225 1635879491225 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
RzRdO_6tQT g1SzIRLQXMM None None 1636361229806 1636361229806 1636361229806 ICLR.cc/2022/Conference/Paper4724/-/Official_Review
4bUxFRU-wKY g1SzIRLQXMM None None 1637203608775 1637203608775 1637203608775 ICLR.cc/2022/Conference/Paper4724/-/Official_Comment
mGieKDTWL9Y g1SzIRLQXMM None None 1637534773054 1637534773054 1637534773054 ICLR.cc/2022/Conference/Paper4724/-/Official_Comment
hDwV3d3GCuM g1SzIRLQXMM None None 1637534895651 1637534895651 1637534895651 ICLR.cc/2022/Conference/Paper4724/-/Official_Comment
tk9p-2lfw5O g1SzIRLQXMM None None 1637535074582 1637535074582 1637535074582 ICLR.cc/2022/

### Get references with other settings

In [22]:
# referent=forum_note.id, original=False
# -> for ICLR 2018 this returns the submission/forum note with little content
# -> here it returns some new note with little content
submission_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=False))
print('#References:', len(submission_references_oF))
assert len(submission_references_oF) == 1

print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oF:
    print_metadata(ref)
    # assert ref.id == forum_note.id
    assert ref.original == forum_note.original

#References: 1
id forum original referent cdate tcdate tmdate invitation
fqNvERVHSh g1SzIRLQXMM -iEEvvwzZ1k g1SzIRLQXMM 1632965333637 1633409729580 1633409729785 ICLR.cc/2022/Conference/-/Blind_Submission


In [23]:
print(submission_references_oF[0])

{'cdate': 1632965333637,
 'content': {'_bibtex': '@inproceedings{\n'
                        'geiger2022wiring,\n'
                        'title={Wiring Up Vision: Minimizing Supervised '
                        'Synaptic Updates Needed to Produce a Primate Ventral '
                        'Stream},\n'
                        'author={Franziska Geiger and Martin Schrimpf and '
                        'Tiago Marques and James J. DiCarlo},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2022},\n'
                        'url={https://openreview.net/forum?id=g1SzIRLQXMM}\n'
                        '}',
             'code_of_ethics': '',
             'resubmission': '',
             'serve_as_reviewer': '',
             'student_author': '',
             'submission_guidelines': '',
             'venue': 'ICLR 2022 Spotlight',
             'venueid': 'ICLR.cc/2022/Conference'},
 'dd

### Seemingly useful but actually confusing

In [24]:
# get venue-specific event types
# the problem is that, unexpectedly, this doesn't return Official_Review
invitations = openreview.tools.iterget_invitations(
        guest_client, regex=f'{VENUE}.*'
)
event_types = set()
for invitation in invitations:
    event_types.add(invitation.id.split('/')[-1])
print(event_types)

{'Recruit_Reviewer_Mentees', 'Recruit_Reviewer_Mentors', 'Recruit_Ethics_Reviewers', 'Recruit_Reviewers', 'Reduced_Load', 'Public_Comment', 'Official_Comment', 'Withdraw', 'Assignment_Recruitment', 'Recruit_Area_Chairs', 'Recruit_Senior_Area_Chairs', 'Decision', 'Withdrawn_Submission', 'Comment', 'Desk_Rejected_Submission', 'Blind_Submission'}
